In [17]:
import requests
import time
import datetime
import pandas as pd

In [45]:
resp = requests.get('https://min-api.cryptocompare.com/data/pricemulti?fsyms=BTC&tsyms=USD')

In [25]:
resp.json()

{'BTC': {'USD': 6738.42}}

In [46]:
resp.json()['BTC']['USD']

6730.13

In [9]:
time.sleep(10)
print('hey')

hey


In [50]:
datetime.datetime.now().strftime('%d-%m-%Y %H:%M:%S')

'27-08-2018 09:17:58'

In [51]:
df = pd.DataFrame(columns=['Date', 'Value'])

In [21]:
df.columns

Index(['Date', 'Value'], dtype='object')

In [52]:
df.head()

,Date,Value


In [44]:
date = datetime.datetime.now().strftime('%d-%m-%Y %H:%M')

In [47]:
price = resp.json()['BTC']['USD']

In [48]:
df = df.append({'Date': date, 'Value': price}, ignore_index=True)

In [49]:
df.head()

,Date,Value
0,27-08-2018 09:16,6729.82
1,27-08-2018 09:16,6730.13


In [60]:
def data_retriever(sleep=30):
    global df
    while True:
        resp = requests.get('https://min-api.cryptocompare.com/data/pricemulti?fsyms=BTC&tsyms=USD')
        price = resp.json()['BTC']['USD']
        date = datetime.datetime.now().strftime('%d-%m-%Y %H:%M')
        df = df.append({'Date': date, 'Value': price}, ignore_index=True)
        print(df.shape)
        df.to_csv('data/btc-daily.csv', index=False)
        time.sleep(sleep)

In [ ]:
data_retriever()

In [62]:
df.shape

(261, 2)

In [63]:
df.head()

,Date,Value
0,27-08-2018 09:22,6726.78
1,27-08-2018 09:22,6726.43
2,27-08-2018 09:23,6723.11
3,27-08-2018 09:24,6722.87
4,27-08-2018 09:24,6722.27


In [64]:
df.tail()

,Date,Value
256,27-08-2018 11:36,6711.93
257,27-08-2018 11:36,6708.67
258,27-08-2018 11:37,6710.56
259,27-08-2018 11:37,6710.36
260,27-08-2018 11:38,6710.46


In [71]:
def count_drop(numbers):
    return len([x for x in numbers if x < 0])

def count_up(numbers):
    return len([x for x in numbers if x > 0])

def actual_calc(row):
    if row['Value'] > row['Open 2']:
        return 0
    return 1

def decider(row):
    if row['Drop 7'] < row['Up 7'] and row['Mean Change 7'] > 0 and row['Change'] >= 0:
        return 1
    elif row['Drop 7'] > row['Up 7'] and row['Mean Change 7'] < 0 and row['Change'] >= 0:
        return 0
    elif row['Drop 7'] < row['Up 7'] and row['Mean Change 7'] > 0 or row['Change'] >= 0:
        return -1
    elif row['Drop 7'] < row['Up 7'] and row['Mean Change 7'] < 0 or row['Change'] < 0:
        return -2
    elif row['Drop 7'] < row['Up 7'] and row['Mean Change 7'] > 0 and row['Change'] < 0:
        return -3
    elif row['Drop 7'] > row['Up 7'] and row['Mean Change 7'] < 0 and row['Change'] < 0:
        return -4
    
def prediction(value):
    if value > 0:
        return 1
    return 0

In [72]:
df['Open 2'] = df['Value'].shift(-1)
df['Max 7'] = df['Value'].rolling(7).max()
df['Min 7'] = df['Value'].rolling(7).min()
df['Change'] = df['Value'] - df['Open 2']
df['Mean Change 7'] = df['Change'].rolling(7).mean()
df['Drop 7'] = df['Change'].rolling(7).apply(count_drop)
df['Up 7'] = df['Change'].rolling(7).apply(count_up)
df['Predict'] = df.apply(decider, axis=1)
df['Prediction'] = df['Predict'].apply(prediction)
df['Actual'] = df.apply(actual_calc, axis=1)

In [73]:
df.head(10)

,Date,Value,Open 2,Max 7,Min 7,Change,Mean Change 7,Drop 7,Up 7,Predict,Prediction,Actual
0,27-08-2018 09:22,6726.78,6726.43,NaN,NaN,0.35,NaN,NaN,NaN,-1.0,0,0
1,27-08-2018 09:22,6726.43,6723.11,NaN,NaN,3.32,NaN,NaN,NaN,-1.0,0,0
2,27-08-2018 09:23,6723.11,6722.87,NaN,NaN,0.24,NaN,NaN,NaN,-1.0,0,0
3,27-08-2018 09:24,6722.87,6722.27,NaN,NaN,0.60,NaN,NaN,NaN,-1.0,0,0
4,27-08-2018 09:24,6722.27,6728.78,NaN,NaN,-6.51,NaN,NaN,NaN,-2.0,0,1
5,27-08-2018 09:25,6728.78,6727.77,NaN,NaN,1.01,NaN,NaN,NaN,-1.0,0,0
6,27-08-2018 09:25,6727.77,6727.12,6728.78,6722.27,0.65,-0.048571,1.0,6.0,-1.0,0,0
7,27-08-2018 09:26,6727.12,6727.48,6728.78,6722.27,-0.36,-0.150000,2.0,5.0,-2.0,0,1
8,27-08-2018 09:26,6727.48,6727.92,6728.78,6722.27,-0.44,-0.687143,3.0,4.0,-2.0,0,1
9,27-08-2018 09:27,6727.92,6727.47,6728.78,6722.27,0.45,-0.657143,3.0,4.0,-1.0,0,0


In [74]:
df[df['Prediction'] != df['Actual']].shape

(194, 12)

In [77]:
df.shape

(254, 12)

In [76]:
df.dropna(inplace=True)

In [78]:
X = df[['Value', 'Max 7', 'Min 7', 'Change', 'Mean Change 7', 'Drop 7', 'Up 7']].values
y = df['Actual'].values

In [83]:
from sklearn.externals import joblib

In [85]:
clf = joblib.load('data/bitcoin-predictor.pkl')

In [86]:
clf.score(X, y)

0.9448818897637795

In [87]:
df['Actual'].value_counts()

0    134
1    120
Name: Actual, dtype: int64